In [1]:
import pandas as pd
import numpy as np
import warnings
import re
import string

import spacy

import gensim
from gensim import corpora

# libraries for visualization
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
review_data= pd.read_csv("Reviews.csv")

print('Total reviews---- ',len(review_data))

print('Unique Products---- ',len(review_data.groupby('ProductId')))

print('Unique Users---- ',len(review_data.groupby('UserId')))

Total reviews----  568454
Unique Products----  74258
Unique Users----  256059


In [3]:
def clean_text(text ): 
    delete_dict = {sp_character: '' for sp_character in string.punctuation} 
    delete_dict[' '] = ' ' 
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    #print('cleaned:'+text1)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and  ( not w.isdigit() and len(w)>3))]) 
    
    return text2.lower()

In [4]:
import nltk
nltk.download('stopwords') # run this one time

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gregormilligan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
review_data.dropna(axis = 0, how ='any',inplace=True) #removing any NAs
review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split())) 

print('-------Dataset --------')
print(review_data['Score'].value_counts())
print(len(review_data))
print('-------------------------')
max_review_data_sentence_length  = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text'] < 100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]
df_sampled = df_short_reviews.groupby('Score').apply(lambda x: x.sample(n=20000)).reset_index(drop = True)

print('No of Short reviews')
print(len(df_short_reviews))



#all_sentences = train_data['text'].tolist() + test_data['text'].tolist()

-------Dataset --------
5    363111
4     80655
1     52264
3     42638
2     29743
Name: Score, dtype: int64
568411
-------------------------
No of Short reviews
373281


In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

# remove stopwords from the text
df_sampled['Text']=df_sampled['Text'].apply(remove_stopwords)

In [7]:
import en_core_web_sm
nlp = en_core_web_sm.load()
nlp.disable_pipes('parser', 'ner')
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']): 
    output = []
    for sent in texts:
        doc = nlp(sent) 
        output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags ])
    return output

In [15]:
text_list=df_sampled['Text'][0:10000].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

lucky stock canisters last summer three left written called company several times dont listen well pity wonderful coffee sticks small wastefulbr started facebook group fans canisters want show numbers hopefully companys attentionbr amazon disallowed first review link group youre interested search nescafe facebook youll please bring back nescafe 61oz canisters
['lucky', 'stock', 'canister', 'last', 'summer', 'company', 'several', 'time', 'pity', 'wonderful', 'coffee', 'stick', 'small', 'wastefulbr', 'facebook', 'group', 'fan', 'canister', 'show', 'number', 'attentionbr', 'amazon', 'first', 'review', 'link', 'group', 'interested', 'search', 'nescafe', 'nescafe', 'oz', 'canister']


In [16]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)
lda_model.print_topics()

[(0,
  '0.056*"taste" + 0.036*"flavor" + 0.029*"good" + 0.019*"product" + 0.014*"chocolate" + 0.011*"brand" + 0.011*"review" + 0.011*"money" + 0.010*"awful" + 0.010*"stuff"'),
 (1,
  '0.019*"plant" + 0.013*"year" + 0.012*"week" + 0.011*"trap" + 0.010*"day" + 0.009*"flower" + 0.009*"greenie" + 0.009*"last" + 0.008*"unhappy" + 0.008*"note"'),
 (2,
  '0.113*"coffee" + 0.017*"kcup" + 0.012*"good" + 0.012*"brand" + 0.012*"water" + 0.010*"cup" + 0.010*"ground" + 0.010*"weak" + 0.009*"cheap" + 0.009*"flavor"'),
 (3,
  '0.040*"chip" + 0.033*"water" + 0.027*"bottle" + 0.020*"coconut" + 0.016*"cereal" + 0.014*"brand" + 0.010*"product" + 0.009*"potato" + 0.009*"size" + 0.008*"case"'),
 (4,
  '0.064*"food" + 0.026*"product" + 0.022*"treat" + 0.019*"dog" + 0.015*"good" + 0.015*"chicken" + 0.013*"ingredient" + 0.012*"organic" + 0.011*"cat" + 0.011*"time"'),
 (5,
  '0.013*"popcorn" + 0.013*"batch" + 0.011*"product" + 0.009*"bread" + 0.009*"time" + 0.008*"vitamin" + 0.007*"honey" + 0.007*"year" + 0.00

In [14]:
# Visualize the topics
#https://github.com/bmabey/pyLDAvis
#https://speakerdeck.com/bmabey/visualizing-topic-models
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.009401  0.070315       1        1  16.793634
8     -0.046209  0.104108       2        1  13.370145
4     -0.021657  0.015339       3        1  12.307963
6     -0.093989  0.036663       4        1  12.143836
1     -0.078227 -0.161009       5        1  10.667765
0      0.172660 -0.043441       6        1   9.297241
9      0.032801 -0.015821       7        1   8.030854
3     -0.012316 -0.026867       8        1   7.243115
7      0.052993  0.048815       9        1   6.952432
2     -0.015455 -0.028103      10        1   3.193017, topic_info=           Term       Freq      Total Category  logprob  loglift
31       coffee  34.000000  34.000000  Default  30.0000  30.0000
241      flavor  21.000000  21.000000  Default  29.0000  29.0000
184         cat   7.000000   7.000000  Default  28.0000  28.0000
145        food  14.000000  14.000000  Default  27.0000  27.0000
20        taste  22.000000  22.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
13         love   0.473373   7.613919  Topic10  -4.9312   0.6664
52         time   0.473373   8.584118  Topic10  -4.9312   0.5464
61    different   0.473373   5.981863  Topic10  -4.9312   0.9076
239  difference   0.473371   3.324533  Topic10  -4.9312   1.4950
44       review   0.473371   9.957114  Topic10  -4.9312   0.3980

[552 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
426       2  0.249072      able
426       3  0.249072      able
426       7  0.249072      able
426       8  0.249072      able
877       6  0.747335  abortion
...     ...       ...       ...
27        7  0.087374      year
27        8  0.087374      year
815       5  0.724172     yeast
369       6  0.565477    yellow
657       5  0.724172      yogi

[824 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 9, 5, 7, 2, 1, 10, 4, 8, 3])